In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualize
import matplotlib.pyplot as plt # visualize

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Content: pour les deux fichiers CSV

catégorique nominale (CN), catégorique ordinale (CO), numérique (numeric)

1. school - student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)
2. sex - student's sex (binary: 'F' - female or 'M' - male)
3. age - student's age (numeric: from 15 to 22)
4. address - student's home address type (binary: 'U' - urban or 'R' - rural)
5. famsize - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)
6. Pstatus - parent's cohabitation status (binary: 'T' - living together or 'A' - apart)
7. Medu - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
8. Fedu - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
9. Mjob - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
10. Fjob - father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
11. reason - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
12. guardian - student's guardian (nominal: 'mother', 'father' or 'other')
13. traveltime - home to school travel time (numeric: 1 - 1 hour)
14. studytime - weekly study time (numeric: 1 - 10 hours)
15. failures - number of past class failures (numeric: n if 1<=n<3, else 4)
16. schoolsup - extra educational support (binary: yes or no)
17. famsup - family educational support (binary: yes or no)
18. paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
19. activities - extra-curricular activities (binary: yes or no)
20. nursery - attended nursery school (binary: yes or no)
21. higher - wants to take higher education (binary: yes or no)
22. internet - Internet access at home (binary: yes or no)
23. romantic - with a romantic relationship (binary: yes or no)
24. famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
25. freetime - free time after school (numeric: from 1 - very low to 5 - very high)
26. goout - going out with friends (numeric: from 1 - very low to 5 - very high)
27. Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
28. Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
29. health - current health status (numeric: from 1 - very bad to 5 - very good)
30. absences - number of school absences (numeric: from 0 to 93)

These grades are related with the course subject, Math or Portuguese:

31. G1 - first period grade (numeric: from 0 to 20)
32. G2 - second period grade (numeric: from 0 to 20)
33. G3 - final grade (numeric: from 0 to 20, output target)



# Survey of students portuguese language courses

In [ ]:
df_por = pd.read_csv("/kaggle/input/student-alcohol-consumption/student-por.csv")

In [ ]:
df_por.head()

In [ ]:
df_por.columns

In [ ]:
df_por.describe()

# Survey of students math language courses

In [ ]:
df_maths = pd.read_csv("/kaggle/input/student-alcohol-consumption/student-mat.csv")

In [ ]:
df_maths.head()

In [ ]:
df_maths.columns

In [ ]:
df_maths.describe()

# Workday alcohol consumption
Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)

In [ ]:
df_por.groupby("Dalc").mean()

In [ ]:
df_maths.groupby("Dalc").mean()

# Weekend alcohol consumption
Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high) 

In [ ]:
df_por.groupby("Walc").mean()

In [ ]:
df_maths.groupby("Walc").mean()

# Students in both courses
Additional note: there are several (382) students that belong to both datasets .
These students can be identified by searching for identical attributes
that characterize each student, as shown in the annexed R file.

In [ ]:
df_class = pd.merge(df_por, df_maths, how='inner', on= ["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"])
print((df_class.shape)) # 382 students

# La consommation d'alcool (frequency)



In [ ]:
df_all  = pd.concat([df_por,df_maths],ignore_index=True).drop_duplicates().reset_index(drop=True)

In [ ]:
df_all.columns

In [ ]:
df_all

In [ ]:
#Les attributs "payé" est spécifique au cours plutôt que spécifique à l'élève, 
#je l'élimine donc de la liste des attributs
#df_all.drop(['paid'], axis=1)

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(df_all.corr(),annot = True,fmt = ".2f",cbar = True)
plt.xticks(rotation=90)
plt.yticks(rotation = 0)

Comme on peut le voir sur la carte de corrélation, seules les notes d'examen sont fortement corrélées les unes avec les autres.
Les étudiants prennent presque la même note à chaque examen.

In [ ]:
# La consommation d'alcool dans la semaine entière : lundi au dimanche
df_all['total_consumption'] = df_all['Dalc'] + df_all['Walc']

In [ ]:
df_all.groupby("total_consumption").mean()

Aucun élève ne consomme pas d'alcool. Tous les élèves consomment au moins 2 fois de l'alcool par semaine.

# La réussite des élèves selon la consommation d'alcool (Gautier)



## Répartition de la consommation d'alcool selon les jours de la semaine dans les deux classes

In [ ]:
dalc_sum = df_all['Dalc'].value_counts(normalize = True)*100
dalc_sum

In [ ]:
# Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high) 
plt.title('Partition de niveau de consommation d\'alcool dans les jours de la semaine dans les 2 classes')
dalc_sum.plot.pie(figsize=(8, 8),autopct = lambda x: str(round(x, 2)) + '%')
plt.legend(["Niveau 1","Niveau 2","Niveau 3","Niveau 4","Niveau 5"])
plt.rcParams['figure.figsize'] = (15, 5)
plt.show()

In [ ]:
walc_sum = df_all['Walc'].value_counts(normalize = True)*100
walc_sum

In [ ]:
# Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high) 
plt.title('Partition de niveau de consommation d\'alcool pendant les week-end dans les 2 classes')
walc_sum.plot.pie(figsize=(8, 8), autopct = lambda x: str(round(x, 2)) + '%')
plt.legend(["Niveau 1","Niveau 2","Niveau 3","Niveau 4","Niveau 5"])
plt.rcParams['figure.figsize'] = (15, 5)
plt.show()

# La consommation d'alcool selon le temps libre (Francis)

In [ ]:
# age et absence
df_all[['age','absences']].head(n=10)

In [ ]:
# le graphe sur la moyenne de la colonne age et absence
means_a_abs = df_all[['age','absences']].groupby(['age']).mean()
means_a_abs.plot.bar()

In [ ]:
# selection des colonnes à étudier
a_trav_fail_free_go_abs = df_all[['age','traveltime','failures','freetime','goout','absences','studytime','total_consumption']]
all_columns = a_trav_fail_free_go_abs.groupby("age").mean()
all_columns

In [ ]:
conso10 = a_trav_fail_free_go_abs[a_trav_fail_free_go_abs['total_consumption'] == 10] 
conso10[['age','absences','freetime','goout','failures']].groupby(['age']).mean().plot.bar(title = " Observation pour une consommation très élevée dans toute la semaine ")

In [ ]:
total_consumption = a_trav_fail_free_go_abs[a_trav_fail_free_go_abs['total_consumption'] < 11] 
age_alc = total_consumption.groupby("total_consumption").mean()
age_alc

In [ ]:
age_alc[['absences','goout','freetime']].plot(kind ='bar',title = "Lien entre les abscences à l'école, temps libre après l'école et les sorties avec les ami(e)s selon la consommation d'alcool dans la semaine")

# La consommation d'alcool selon la situation familiale (Laura)

In [ ]:
df_all_family = df_all[['school', 'address', 'famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','guardian','famsup','nursery','higher','internet','famrel','Dalc','Walc','total_consumption' ]]
df_all_family

In [ ]:
# Medu - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
plt.title("Partition des élèves les 2 classes selon l'éducation de la mère")
df_all.Medu.value_counts(normalize=True).plot.pie(figsize=(5, 5), autopct = lambda x: str(round(x, 2)) + '%')
plt.legend(["none", "primary education (4th grade)", "5th to 9th grade", "2nd education","higher education"])
plt.rcParams['figure.figsize'] = (15, 5)
plt.show()

In [ ]:
medu_df = df_por.groupby(['Medu']).mean()
plt.rcParams['figure.figsize'] = (20, 10)
medu_df[['G1','G2','G3']].plot()

In [ ]:
# Fedu - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
plt.title("Partition des élèves les 2 classes selon l'éducation du père")
df_all.Fedu.value_counts(normalize=True).plot.pie(figsize=(5, 5), autopct = lambda x: str(round(x, 2)) + '%')
plt.legend(["none", "primary education (4th grade)", "5th to 9th grade", "2nd education","higher education"])
plt.rcParams['figure.figsize'] = (15, 5)
plt.show()

In [ ]:
fedu_df = df_por.groupby(['Fedu']).mean()
plt.rcParams['figure.figsize'] = (20, 10)
fedu_df[['G1','G2','G3']].plot()

In [ ]:
# Mjob - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other') 
plt.title("Partition des élèves les 2 classes selon le travail de la mère")
df_all.Mjob.value_counts(normalize=True).plot.pie(figsize=(5, 5), autopct = lambda x: str(round(x, 2)) + '%')
plt.legend(['teacher', 'health','services', 'at_home', 'other'])
plt.rcParams['figure.figsize'] = (15, 5)
plt.show()

In [ ]:
mjob_df = df_all.groupby(['Mjob']).mean()
plt.rcParams['figure.figsize'] = (20, 10)
mjob_df[['G1','G2','G3']].plot(kind='bar')

In [ ]:
# Fjob - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other') 
plt.title("Partition des élèves les 2 classes selon le travail de la père")
df_all.Fjob.value_counts(normalize=True).plot.pie(figsize=(5, 5), autopct = lambda x: str(round(x, 2)) + '%')
plt.legend(['teacher', 'health','services', 'at_home', 'other'])
plt.rcParams['figure.figsize'] = (15, 5)
plt.show()

In [ ]:
fjob_df = df_all.groupby(['Fjob']).mean()
plt.rcParams['figure.figsize'] = (20, 10)
fjob_df[['G1','G2','G3']].plot(kind='bar')

In [ ]:
# famsup - family educational support (binary: yes or no) 
plt.title("Partition des élèves les 2 classes selon s'il existe un soutien éducatif familial")
df_all.famsup.value_counts(normalize=True).plot.pie(figsize=(5, 5), autopct = lambda x: str(round(x, 2)) + '%')
plt.legend(['yes', 'no'])
plt.rcParams['figure.figsize'] = (15, 5)
plt.show()

# La consommation d'alcool selon le genre et la santé (Renata)

In [ ]:
rural_data = df_all[df_all['address']== 'R']
urban_data = df_all[df_all['address']== 'U']

In [ ]:
#rural area
rural_df = rural_data[['sex', 'total_consumption']]
rural_df_h = rural_df[rural_df['sex'] == 'M']
rural_df_h = rural_df_h.groupby(['total_consumption']).count()
rural_df_h.plot(kind='bar', title = "Consommation d'alcool chez les hommes")

In [ ]:
rural_df = rural_data[['sex', 'total_consumption']]
rural_df_f = rural_df[rural_df['sex'] == 'F']
rural_df_f = rural_df_f.groupby(['total_consumption']).count()
rural_df_f.plot(kind='bar', title = "Consommation d'alcool chez les femmes")

In [ ]:
df_all = df_all[['sex', 'total_consumption']]
df_all_h = df_all[df_all['sex'] == 'M']
df_all_h = df_all.groupby(['total_consumption']).count()
df_all_h.plot(kind='bar', title = "Consommation d'alcool chez les hommes")

In [ ]:
df_all = df_all[['sex', 'total_consumption']]
df_all_f = df_all[df_all['sex'] == 'F']
df_all_f = df_all.groupby(['total_consumption']).count()
df_all_f.plot(kind='bar', title = "Consommation d'alcool chez les femmes")

# Conclusion des observations